The code in this file trains the an efficientnet model on the train data

In [ ]:
# Install the efficientnet model
!pip install efficientnet -q

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import KFold
import re
import tensorflow_addons as tfa

ModuleNotFoundError: No module named 'efficientnet'

In [2]:
# Initialize the TPU
def select_tpu():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy
select_tpu()

NameError: name 'tf' is not defined

In [ ]:
# Function to read the jpeg images, reshape them, and convert them into float
def decode_image(image_data):
        
    image = tf.image.decode_jpeg(image_data, channels=3)
    
    image = tf.reshape(image, [256, 256, 3])
    image = tf.cast(image, tf.float32) / 255    
    return image


In [ ]:
# Mapping of the tfrecord fields to corresponding datatype
feature_map = {
    'image':tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.int64),
    'StudyInstanceUID': tf.io.FixedLenFeature([], tf.string),
    'only_label': tf.io.FixedLenFeature([], tf.int64),
    'Negative for Pneumonia': tf.io.FixedLenFeature([], tf.int64),
    'Typical Appearance': tf.io.FixedLenFeature([], tf.int64),
    'Indeterminate Appearance': tf.io.FixedLenFeature([], tf.int64),
    'Atypical Appearance': tf.io.FixedLenFeature([], tf.int64),
    'img_path':tf.io.FixedLenFeature([], tf.string),
    'image_id':tf.io.FixedLenFeature([], tf.string),
    'dim0':tf.io.FixedLenFeature([], tf.int64),
    'dim1':tf.io.FixedLenFeature([], tf.int64),
    'xmin':tf.io.FixedLenFeature([], tf.float32),
    'ymin':tf.io.FixedLenFeature([], tf.float32),
    'xmax':tf.io.FixedLenFeature([], tf.float32),
    'ymax':tf.io.FixedLenFeature([], tf.float32)
}

In [ ]:
# Function to read the tfrecords
def read_tfrecord(example, labeled=True):
    '''
        example: single tfrecord
        labeled: indicated whether the data is labeled or unlabeled
        
        returns:
            image: An image tensor
    '''
    
    # Parse an record with the help of the feature map
    example = tf.io.parse_single_example(example, feature_map)  
    
    # Decode the image from the record
    image = decode_image(example['image'])  
    
    # If the data is labeled, return the multiple labels along with
    # the image data
    if labeled:
        label = tf.one_hot(example['target'], 4)
        
    else:
        return image
    return image, label

In [ ]:
# Function that returns a tf dataset from the list of files
def get_dataset(files,batch_size=10,shuffle = False, repeat = False, labeled=True):
    
    AUTO     = tf.data.experimental.AUTOTUNE
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    
    
    if labeled:
        ds = ds.map(lambda example:read_tfrecord(example), num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_tfrecord(example, labeled=False), 
                    num_parallel_calls=AUTO)  
      
    #ds = ds.map(lambda img, imgname_or_label: (decode_image(img),imgname_or_label),num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTO)
    return ds

In [ ]:
data=pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
study_labels=list(data.columns[1:5])
study_labels

In [ ]:
# Function that returns an efficientnet model for the transfer learning
def get_model():
    
    inp = tf.keras.layers.Input(shape=(256,256,3))
    base = efn.EfficientNetB6(input_shape=(256,256,3),weights='imagenet',include_top=False)
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    x = tf.keras.layers.Dense(4,activation='softmax')(x)
    model = tf.keras.Model(inputs=inp,outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01)
    auc = tf.keras.metrics.AUC(curve='ROC',
                               multi_label=True)
    acc = tf.keras.metrics.CategoricalAccuracy()
    f1  = tfa.metrics.F1Score(num_classes=4,average='macro',threshold=None)
    model.compile(optimizer=opt,loss=loss,metrics=[auc, acc, f1])
    return model
    
       
    
    
    
   

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
# Read the tfrec files
GCS_PATH=KaggleDatasets().get_gcs_path('tfrecssiim')
train_files=np.sort(np.array(tf.io.gfile.glob(GCS_PATH+'/train*.tfrec')))
test_files=np.sort(np.array(tf.io.gfile.glob(GCS_PATH+'/test*.tfrec')))

In [ ]:
# Count thefiles in the tfrecord
def get_file_count(ds):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in ds]
    
    return np.sum(n)

In [ ]:
FOLDS=5
SEED=42
EPOCHS=20
BATCH_SIZE=16


In [ ]:
def get_lr_callback(batch_size=8):
    lr_start   = 0.000005
    lr_max     = 0.00000125 * 8 * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr
   

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

#_=get_lr_callback(BATCH_SIZE[0], plot=True )

In [ ]:
# 

skf=KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
preds=np.zeros((get_file_count(test_files),1))
files_train=[]
files_val=[]
oof_pred=[]
oof_tar=[]
for fold, (tridx, validx) in enumerate(skf.split(np.arange(15))):
    print(f'Running fold {fold}')
    files_train=tf.io.gfile.glob([GCS_PATH + '/train%.2i*.tfrec'%i for i in tridx])
    files_val=tf.io.gfile.glob([GCS_PATH+'/train%.2i*.tfrec'% i for i in validx])
    
    with select_tpu().scope():
        model=get_model()
        
        
        # save the best model in each fold
        cb=tf.keras.callbacks.ModelCheckpoint('fold-%i.h5'%fold, monitor='val_loss', verbose=0, save_best_only=True, 
                                             save_weights_only=True, mode='min', save_freq='epoch')
        # Train the model
        steps_per_epoch=get_file_count(train_files)/BATCH_SIZE//8
        print(steps_per_epoch)
           
        # Create the train and val datasets
        tr_ds=get_dataset(files_train,BATCH_SIZE, shuffle = True, repeat = True)
        val_ds=get_dataset(files_val,BATCH_SIZE, shuffle = False, repeat = False)
        
        # Train the model
        history=model.fit(tr_ds, epochs=EPOCHS, callbacks=[cb,get_lr_callback(BATCH_SIZE)],steps_per_epoch=steps_per_epoch,validation_data=val_ds, verbose=1)
        
        
        
    
        

In [ ]:
model.summary()